In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset=pd.read_csv("../input/predicting-churn-for-bank-customers/Churn_Modelling.csv")

x=dataset.iloc[:,3:13]
y=dataset.iloc[:,13]

In [ ]:
x

In [ ]:
y

In [ ]:
x.info()

# Lets Encode Geography And Gender

In [ ]:
geography=pd.get_dummies(x["Geography"],drop_first=True)
gender=pd.get_dummies(x["Gender"],drop_first=True)

In [ ]:
x=pd.concat([x,geography,gender],axis=1)

In [ ]:
## Drop Unnecessary columns
x=x.drop(['Geography','Gender'],axis=1)

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

# In ANN feature scaling is very important so that all inputs are at a comparable range and only the weights assigned to them are, in fact, the only factor which makes a difference on the predicted value.

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
#scales test train to float data type from unknown datatype

# Lets Make ANN

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout

Randomly initialise the weights with small numbers close to zero but not zero. This will be done by our Dense function.

Distribute features of the first observation, from your dataset, per each node in the input layer. Thus, eleven independent variables will be added to our input layer.

In [ ]:
# Initialising the ANN
classifier = Sequential()

Adding the input layer

units - number of nodes to add to the hidden layer.

Tip: units should be the average of nodes in the input layer (11 nodes) and the number of nodes in the output layer (1 node). For this case is 11+1/2 = 6

kernel_initializer - randomly initialize the weight with small numbers close to zero, according to uniform distribution.

activation - Activation function.

input_dim - number of nodes in the input layer, that our hidden layer should be expecting

In [ ]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'he_uniform', activation = 'relu', input_dim = 11 ))

In [ ]:
# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'he_uniform', activation = 'relu'))

**To define the first hidden layer, we firstly will have to define an activation function. The best one is the Rectifier Function and we’ll choose this one for the hidden layers. Furthermore, also by using a Sigmoid function to the output layer will allow us to calculate the probabilities of the different class (leaving or staying the bank). In the end, we will be able to rank the customers by their probability to leave the bank.**

In [ ]:
# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))

Cost Function : Measure the generated error by comparing the predicted value with the true value.

Compiling the ANN -

optimizer — algorithm to use to find the best weights that will make our system powerful

loss — Loss function within our optimizer algorithm

metric — criteria to evaluate the model

In [ ]:
# Compiling the ANN
classifier.compile(optimizer='adam',loss="binary_crossentropy",metrics=["accuracy"])

Back-Propagation: from the output to the input layer, the calculated error is back-propagated and the weights get updated according to the influence they had on the error. The learning rates indicate how much these weights are updated.

Reinforcement Learning : Update weights at each observation or Batch Learning : Update the weights after each batch of observations

When the system has gone through the whole training dataset, an epoch has been run.

In [ ]:
# Fitting the ANN to the Training set
classifier.fit(x_train, y_train, batch_size = 10, epochs = 100  )

batch_size : number of observations after which we update the weights

epochs: How many times you train your model

# Prediction Time !

In [ ]:
#Predicting the Test set results
y_pred = classifier.predict(x_test)

#Threshold of 50%
y_pred = (y_pred>0.5)

y_pred

**According to our model the first five customer will not leave the bank while the sixth on the rank will.**

In [ ]:
#Making the COnfusion Matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

In [ ]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred,y_test)
score